## Library imports

In [1]:
import numpy as np
import pandas as pd
from chromatography import *
from torch import optim, tensor
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mayavi import mlab
%matplotlib qt

## Dataset Import

In [9]:
sample = 'Peter32'
alist = pd.read_csv(f'../data/{sample}.csv')
# GilarSample
# Peterpeptides
# Roca
# Peter32
# Eosin
# Alizarin
# Controlmix1
# Controlmix2
#alist['k0'] = np.exp(alist.lnk0)

## Useful functions definition

In [3]:
def step_decay(lr, iteration, num_episodes, steps=10, decay_factor=0.8):
    if iteration % (num_episodes // steps) == 0:
        return lr * decay_factor
    
    return lr

In [4]:
def loss_field(exp, taus, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], taus)
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

In [5]:
# Parameters
run_time_lim = 10.0
sigma_max = 0.2
taus = [.25, .25, 3000000]
num_episodes=5000
batch_size=10
optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65)
lr=.1
print_every=100
lr_decay=lambda a, b, c: step_decay(a, b, c, steps=7, decay_factor=0.5)
baseline=0.55
max_norm = 2.
beta = .0
run = 1

In [ ]:
# Parameters
run_time_lim = 10.0
sigma_max = 0.2
taus = [.5, 3000000]
num_episodes=5000
batch_size=10
optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65)
lr=.1
print_every=100
lr_decay=lambda a, b, c: step_decay(a, b, c, steps=7, decay_factor=0.5)
baseline=0.55
max_norm = 2.
beta = .0
run = 2

In [ ]:
# Parameters
run_time_lim = 10.0
sigma_max = 0.2
taus = [.5, 3000000]
num_episodes=5000
batch_size=10
optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65)
lr=.1
print_every=100
lr_decay=lambda a, b, c: step_decay(a, b, c, steps=7, decay_factor=0.5)
baseline=0.55
max_norm = 2.
beta = .0
run = 3
lim = 0.2
sample += "_ISO"

In [ ]:
# Parameters
run_time_lim = 10.0
sigma_max = 0.3
taus = [.5, 3000000]
num_episodes=5000
batch_size=10
optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65)
lr=.1
print_every=100
lr_decay=lambda a, b, c: step_decay(a, b, c, steps=7, decay_factor=0.5)
baseline=0.55
max_norm = 2.
beta = 0.
run = 4


In [11]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, run_time=10.0)
pol = Policy(len(taus), sigma_max = sigma_max)
losses, best_program, mus, sigmas, n_pars, samples, mu_grads, sigma_grads = reinforce(
        exp, 
        pol, 
        taus, 
        num_episodes=num_episodes, 
        batch_size=batch_size,
        optim=optim,
        lr=lr, 
        print_every=print_every,
        lr_decay=lr_decay,
        baseline=baseline,
        max_norm=max_norm,
        beta=beta,
        #lim=lim
    )

Loss: 0.7969264976876692, epoch: 100/5000
Loss: 0.6233829924118557, epoch: 200/5000
Loss: 0.5960820220758902, epoch: 300/5000
Loss: 0.5802556149174862, epoch: 400/5000
Loss: 0.5586815957003018, epoch: 500/5000
Loss: 0.5495044924498711, epoch: 600/5000
Loss: 0.542914272665019, epoch: 700/5000
Loss: 0.5393397128503807, epoch: 800/5000
Loss: 0.5360148274655442, epoch: 900/5000
Loss: 0.5348675724136648, epoch: 1000/5000
Loss: 0.5336000376101926, epoch: 1100/5000
Loss: 0.533003992362425, epoch: 1200/5000
Loss: 0.5312876043863749, epoch: 1300/5000
Loss: 0.530634497843125, epoch: 1400/5000
Loss: 0.5295334494366204, epoch: 1500/5000
Loss: 0.5302265368477906, epoch: 1600/5000
Loss: 0.5290294712210267, epoch: 1700/5000
Loss: 0.5291585455675863, epoch: 1800/5000
Loss: 0.5289619292628926, epoch: 1900/5000
Loss: 0.528574200019023, epoch: 2000/5000
Loss: 0.5284410111386806, epoch: 2100/5000
Loss: 0.5282242566691738, epoch: 2200/5000
Loss: 0.5277255881519449, epoch: 2300/5000
Loss: 0.5277446644982894

In [8]:
exp.reset()
exp.run_all(pol.mu.detach().numpy(), taus)
exp.print_analytes(rc=(7, 5), angle=45,  title=f'Solvent Strength function\nLoss: {round(exp.loss(),4)}')
plt.show()
plt.legend()
#plt.savefig(f'results/{sample}_result_{run}.png')

In [ ]:
exp.reset()
exp.run_all(best_program.numpy(), taus)
exp.print_analytes(rc=(7, 5), angle=45,  title=f'Solvent Strength function\nLoss: {round(exp.loss(),4)}')
plt.legend()
plt.savefig(f'results/{sample}_best_result_{run}.png')

In [ ]:
plt.plot(mus[:, 0], label='Mu: phi1')
plt.plot(mus[:, 1], label='Mu: phi2')
#plt.plot(mus[:, 2], label='Mu: phi3')
#plt.plot(mus[:, 3], label='Mu: phi4')
plt.ylim((0,1))
plt.legend()
plt.savefig(f'results/{sample}_mus_{run}.png')

In [ ]:
plt.plot(sigmas[:, 0], label='Sigma: phi1')
plt.plot(sigmas[:, 1], label='Sigma: phi2')
#plt.plot(sigmas[:, 2], label='Sigma: phi3')
#plt.plot(sigmas[:, 3], label='Sigma: phi4')
plt.ylim((0,sigma_max))
plt.legend()
plt.savefig(f'results/{sample}_sigmas_{run}.png')

In [ ]:
plt.plot(np.arange(print_every, num_episodes + print_every, print_every), losses)
plt.xlabel("Episodes")
plt.ylabel("Loss")
plt.title("Loss during learning process")
plt.savefig(f'results/{sample}_loss_{run}.png')

In [12]:
X, Y, Loss_field = loss_field(exp, [.25, 30000], N = 500)
plt.contourf(X, Y, Loss_field, levels=100)
plt.xlabel("Phi1")
plt.ylabel("Phi2")
index = np.unravel_index(np.argmin(Loss_field), Loss_field.shape)
plt.scatter(X[index], Y[index], c="r", s=5)
plt.title(f"Loss Field, change of phi at 0.5\nGlobal min ({round(np.min(Loss_field), 3)}) at {round(X[index], 4), round(Y[index], 4)}, red dot")
#plt.savefig(f'results/{sample}_loss_filed_at_5.png')

Text(0.5, 1.0, 'Loss Field, change of phi at 0.5\nGlobal min (0.471) at (0.3747, 0.1984), red dot')

In [ ]:
fig, ax = plt.subplots()
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()

line, = plt.plot(mus[:, 0], mus[:, 1], color='k')
scat = plt.scatter(samples[:, 0], samples[:, 1], c='r', s=3)

def update(num, mu, samples, line, scat):
    line.set_data(mu[num-2:num, 0], mu[num-2:num, 1])
    line.axes.axis([0, 1, 0, 1])
    scat.set_offsets(samples[(num-2)*10: num*10 - 10, :])
    return line,

ani = animation.FuncAnimation(fig, update, len(mus[:, 0]), fargs=[mus, samples, line, scat],
                              interval=10, blit=False)
plt.show()

In [ ]:
import matplotlib.patches as patches
fig, ax = plt.subplots()
ax.set_ylim(0, 1)
ax.set_xlim(0, 1)
plt.xlabel("phi1")

plt.ylabel("phi2")
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()

n = 1998
patch = patches.Arrow(mus[n, 0], mus[n, 1], mu_grads[n, 0], mu_grads[n, 1], width=0.02, color='k')
ax.add_patch(patch)
plt.scatter(samples[n*10:n*10+10, 0], samples[n*10:n*10+10:, 1], c='r', s=5, label='Samples')
plt.scatter(mus[n, 0], mus[n, 1], c='b', s=20, label='current mu')
plt.scatter(mus[n + 1, 0], mus[n + 1, 1], c='w', s=20, label='next mu')
plt.title(f"sigmas:{sigmas[n,:]}")
plt.legend()

In [ ]:
import matplotlib.patches as patches

fig, ax = plt.subplots()
ax.set_ylim(0, 1)
ax.set_xlim(0, 1)
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()

patch = patches.Arrow(mus[0, 0], mus[0, 1], mu_grads[0, 0], mu_grads[0, 1], width=0.02, color='k')
scat1 = plt.scatter(samples[:, 0], samples[:, 1], c='r', s=5)
scat2 = plt.scatter(mus[:, 0], mus[:, 1], c='b', s=20)
def init():
    ax.add_patch(patch)
    return patch,

def update(num, mu, samples, scat1, scat2, ax):
    ax.patches.pop(0)
    patch = patches.Arrow(mus[num-2, 0], mus[num-2, 1], mu_grads[num-2, 0], mu_grads[num-2, 1], width=0.01, color='k')
    ax.add_patch(patch)
    scat1.set_offsets(samples[(num-2)*10: num*10, :])
    scat2.set_offsets(mu[num-2: num, :])
    return scat1,

ani = animation.FuncAnimation(fig, update, len(mus[:, 0]), fargs=[mus, samples, scat1, scat2, ax],
                              interval=10000, blit=False, init_func=init)


In [ ]:
N = -1
mlab.surf(X.T[:N, :N], Y.T[:N, :N], Loss_field.T[:N, :N])

In [ ]:
n=3
plt.contourf(X, Y, Loss_field, levels=100)
plt.scatter(samples[n*10:10+10*n ,0], samples[n*10:10+10*n,1], s=1.,c='r')
plt.arrow(mus[n,0], mus[n,1], mu_grads[n,0], mu_grads[n,1], length_includes_head=True, head_width=0.04, head_length=0.2)
plt.scatter(mus[n:n+2, 0], mus[n:n+2, 1])
plt.xlim((0, 1))
plt.ylim((0, 1))

In [ ]:
phis = np.linspace(0, 1, 1000)
losses = []
for phi1 in phis:
    exp.reset()
    exp.run_all([0.2, phi1], [0.25, 3000000])
    losses.append(exp.loss())
plt.plot(losses[:])

In [ ]:
plt.contourf(X, Y, Loss_field, levels=100)

In [ ]:
exp_time = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=10.0)
pol_time = PolicyTime(2, sigma_max = .2)
losses_time, best_program_time, mus_time, sigmas_time, n_par_time = reinforce_delta_tau(
        exp_time, 
        pol_time,
        num_episodes=5000, 
        batch_size=10,
        optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65),
        lr=.1, 
        print_every=100,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=5, decay_factor=0.5),
        baseline=0.65,
        max_norm=2.
    )

In [ ]:
exp_time.reset()
exp_time.run_all(pol_time.mu.detach().numpy()[:2], [0.25, 3000000])
exp_time.print_analytes(rc=(7, 5))
exp_time.loss()

In [ ]:
exp_time.reset()
exp_time.run_all(best_program_time[0], best_program_time[1])
exp_time.print_analytes(rc=(7, 5))
exp_time.loss()

In [ ]:
plt.plot(losses_time)

In [ ]:
plt.plot(mus_time[:, 0], label='Mu: 0')
plt.plot(mus_time[:, 1], label='Mu: 1')
plt.plot(mus_time[:, 2], label='Mu: 2')
plt.legend()
plt.show()

In [ ]:
plt.plot(sigmas_time[:, 0], label='Sigma: 0')
plt.plot(sigmas_time[:, 1], label='Sigma: 1')
plt.plot(sigmas_time[:, 2], label='Sigma: 2')
plt.plot(sigmas_time[:, 3], label='Sigma: 3')
plt.plot(sigmas_time[:, 4], label='Sigma: 4')
plt.plot(sigmas_time[:, 5], label='Sigma: 5')
plt.plot(sigmas_time[:, 6], label='Sigma: 6')

plt.legend()
plt.show()